In [14]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
## setup logging and infinstor platform callback

import logging

import pkg_resources
# when the cell is run, it looks like logging.info() or similar is called (by infinstor platform code or a library) before the call below.  as a result, basicConfig() is ignored (don't see the timestamps because of this)
# enabling logging.DEBUG is useful to see output from urllib3, boto3 and other libraries.  
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(process)d - %(name)s - %(levelname)s - %(message)s")

logging.info("first line of multilex_NERModel_lg_xform")

import os
from pathlib import Path
def infin_transform_all_objects(input_dir, output_dir, **kwargs):
    logging.info("input_dir=" + input_dir + ", output_dir=" + output_dir)
    # onlyfiles = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
    # for f in onlyfiles:
    #  logging.info("file in directory %s = %s. size = %d", input_dir, os.path.join(input_dir, f), Path(os.path.join(input_dir, f)).stat().st_size)
    for path, subdirs, files in os.walk(input_dir):
        print(path)
        for name in files:
            logging.info("file in directory %s = %s. size = %d", path, os.path.join(path, name), Path(os.path.join(path, name)).stat().st_size)

    NERModel_lg(input_dir, output_dir)


# %%
import subprocess
import sys
def NERModel_lg(input_dir, output_dir):
        import os
        import torch
        import time
        from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

        from transformers import (
            AlbertConfig,
            AlbertForQuestionAnswering,
            AlbertTokenizer,
            squad_convert_examples_to_features
        )

        from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

        from transformers.data.metrics.squad_metrics import compute_predictions_logits

        # READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
        use_own_model = False

        if use_own_model:
          model_name_or_path = "/content/model_output"
        else:
          model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

        output_dir = ""

        # Config
        n_best_size = 1
        max_answer_length = 30
        do_lower_case = True
        null_score_diff_threshold = 0.0

        def to_list(tensor):
            return tensor.detach().cpu().tolist()

        # Setup model
        config_class, model_class, tokenizer_class = (
            AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
        config = config_class.from_pretrained(model_name_or_path)
        tokenizer = tokenizer_class.from_pretrained(
            model_name_or_path, do_lower_case=True)
        model = model_class.from_pretrained(model_name_or_path, config=config)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model.to(device)

        processor = SquadV2Processor()

        def run_prediction(question_texts, context_text):
            """Setup function to compute predictions"""
            examples = []

            for i, question_text in enumerate(question_texts):
                example = SquadExample(
                    qas_id=str(i),
                    question_text=question_text,
                    context_text=context_text,
                    answer_text=None,
                    start_position_character=None,
                    title="Predict",
                    is_impossible=False,
                    answers=None,
                )

                examples.append(example)

            features, dataset = squad_convert_examples_to_features(
                examples=examples,
                tokenizer=tokenizer,
                max_seq_length=384,
                doc_stride=128,
                max_query_length=64,
                is_training=False,
                return_dataset="pt",
                threads=1,
            )

            eval_sampler = SequentialSampler(dataset)
            eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

            all_results = []

            for batch in eval_dataloader:
                model.eval()
                batch = tuple(t.to(device) for t in batch)

                with torch.no_grad():
                    inputs = {
                        "input_ids": batch[0],
                        "attention_mask": batch[1],
                        "token_type_ids": batch[2],
                    }

                    example_indices = batch[3]

                    outputs = model(**inputs)

                    for i, example_index in enumerate(example_indices):
                        eval_feature = features[example_index.item()]
                        unique_id = int(eval_feature.unique_id)

                        output = [to_list(output[i]) for output in outputs]

                        start_logits, end_logits = output
                        result = SquadResult(unique_id, start_logits, end_logits)
                        all_results.append(result)

            output_prediction_file = "predictions.json"
            output_nbest_file = "nbest_predictions.json"
            output_null_log_odds_file = "null_predictions.json"

            predictions = compute_predictions_logits(
                examples,
                features,
                all_results,
                n_best_size,
                max_answer_length,
                do_lower_case,
                output_prediction_file,
                output_nbest_file,
                output_null_log_odds_file,
                False,  # verbose_logging
                True,  # version_2_with_negative
                null_score_diff_threshold,
                tokenizer,
            )

            return predictions


        #we have to read csv here!
        import pandas as pd 
        
#         input_file_fullpath = os.path.join(input_dir, 'infinstor','todays_report.csv')
        input_file_fullpath = os.path.join(input_dir,'todays_report.csv')
        logging.info("reading input artifact " + input_file_fullpath)
        data = pd.read_csv(input_file_fullpath)
        logging.info("completed reading input artifact " + input_file_fullpath)
        
        context = data['title']
        cmp = []
        for i in context:
        #     context = """By Reuters Staff3 Min Readshanghai March 20 Reuters China S Top Securities Regulator On Saturday Urged Underwriters To Tighten Scrutiny On Companies Seeking To List Their Shares Vowing To Punish Those Trying To Bring Sick Companies To The Initial Public Offering Ipo Market Yi Huiman Chairman Of The China Securities Regulatory Commission Csrc Told A Forum That Recent China Stock Market Volatility Is Natural And Risks Controllable But Cautioned Against Harmful Foreign Hot Money Flows Yi S Comments Come Amid Rising Fluctuations In Domestic And Global Stock Markets As Well As Signs Chinese Regulators Are Tightening The Screws On Ipo Approvals The Registration Based Ipo System Doesn T Mean Looser Vetting Requirements Yi Said In The Speech Which Was Published On Csrc S Website It Means Providing Investors With Investable Companies That Have More Value So Requirements On Gatekeepers Are Actually Higher He Said China Has Adopted A U S Style Registration Based Ipo System On Shanghai S Nasdaq Style Star Market And Shenzhen S Start Up Board Chinext In A Bold Reform Designed To Give Market A Bigger Role In Evaluating Ipo Candidates But Since December Stock Exchanges Have Stepped Up Ipo Inspections Leading To A Growing Number Of Companies Cancelling Their Ipo Plans To Justify The Move Yi Said That China Has The World S Biggest Retail Investor Base Of 180 Million So Regulators Need To Make Sure Listing Candidates Make Full And High Quality Disclosures And Comply With China S Industrial Policies Many Underwriters Wear New Shoes But Walk On The Old Path And They Must Step Up Their Due Diligence And Shoulder More Responsibility Yi Said Commenting On Stock Market Volatility Yi Said That Risk Is Controllable As Current Leverage In China S A Share Market Is Not Excessive However He Said China Should Strictly Control Massive Inflows And Outflows Of Hot Money While Continuing To Encourage Normal Cross Border Liquidity Flows For Any Market Big Inflows And Outflows Of Hot Money Would Be Detrimental To Healthy Development And Must Be Severely Controlled Yi Said Reporting By Samuel Shen And Emily Chow Our Standards The Thomson Reuters Trust Principles 
        #     """
            questions = ["which company is going to be listed?","which company is going to be IPO?","which company is going to be public?","which company is going to be listed on stock exchange?","which company is about to launch initial public offering?","which company is about to launch IPO?","which company is it talking about?"]

        #     questions = ["where is the company located?","in which country is it?"]
            # Run method
            predictions = run_prediction(questions, i)

            # Print results
            import statistics as st
            prelist = [i for i in predictions.values() if i != '']

            try:
                    sp = st.mode(prelist)
            except:
                sp = prelist

            cmp.append(sp)


        import pandas as pd
        import time

        
        links1 = data["link"]

        coun = []
        con = []
        body = [] 
        link = []
        date = []
        from newspaper import Article
        for ur in links1:
          try:
            article = Article(ur)
            article.download()
            article.parse()
            print(article.text)
            body.append(article.text)
            link.append(ur)
            date.append(article.publish_date)

          except:
            continue
          time.sleep(0.5)

        for i in range(len(body)):
          if body[i].startswith("We have reserved some of the more detailed information for our registered users. Please sign in if You have") :
            body[i] = " "
        

        #we will use text column for this 

        import pycountry

        text = data['text']
        for j in text:
            for country in pycountry.countries:
                j = str(j)
                if country.name in j:
                    coun.append(country.name)


            questions1 = ["which stock exchange are we talking about?","Where is it going to be listed?", "Where does the trading start?"]
            predictions1 = run_prediction(questions1, j)

            # Print results
            import statistics as st
            prelist = [j for j in predictions.values() if j != '']

            try:
                coun1= st.mode(prelist)
            except:
                coun1= prelist

            con.append(coun1)
        
        dic1 = {"text":body, 'Company Name':cmp,'Country':coun, 'listing':con}
#         print(dic1)
        df = pd.DataFrame.from_dict(dic1,orient='index')
        df = df.transpose()
        df_final = pd.concat([data,df],axis=1)
        df_final=df_final.reset_index(drop=True)
#         df_final.to_csv('EDI_PREIPO_report.csv',index=False)

#         df = pd.DataFrame({"text":body, 'Company Name':cmp,'Country':coun, 'listing':con})
#         df_final = pd.concat([data,df])
        
        
#         df_final.to_csv('EDI_PREIPO_report.csv',index=False)


        # %%

        # import ipynb
    
        
        edi_preipo_report_fname = os.path.join( output_dir, 'EDI_PREIPO_report.csv' )
        logging.info("writing output artifact " + edi_preipo_report_fname + " to " + output_dir)
        df_final.to_csv(edi_preipo_report_fname,index=False)
        logging.info("completed writing output artifact " + edi_preipo_report_fname + " to " + output_dir)


        # %%
        # from ipynb.fs.full.Data_preprocessing import CleanedReport,CleaningData


        # %%
        # CleanedReport('EDI_PREIPO_report_1.csv')


        # %%

NERModel_lg("/home/prachi_multilex", "/home/prachi_multilex")     # uncomment this line to run this as a python script
logging.info("last line of multilex_NERModel_lg_xform")




2021-05-25 13:38:50,274 - 1620309 - root - INFO - first line of multilex_NERModel_lg_xform
2021-05-25 13:38:50,954 - 1620309 - filelock - INFO - Lock 139914510708304 acquired on /home/prachi_multilex/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f.lock
2021-05-25 13:38:50,965 - 1620309 - filelock - INFO - Lock 139914510708304 released on /home/prachi_multilex/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f.lock
2021-05-25 13:38:50,978 - 1620309 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/ktrapeznikov/albert-xlarge-v2-squad-v2/config.json from cache at /home/prachi_multilex/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adf

Futures and Commodity Market News Digital Brands Group, Inc. Launches Innovative Bespoke Apparel Business After Completing $10 Million IPO (NasdaqGS: DBGI)

May 24, 2021 (AB Digital via COMTEX) --

There's an innovative company where technology, innovation, and fashion connect. Meet Digital Brands Group, Inc. (NasdaqGS: DBGI), an innovative, forward-thinking digitally-focused apparel company that is changing the rules for how clothing is offered and sold. And the great news is that, for investors, its business model is designed to deliver maximum profits through its low overhead, high margin business model. Their mission is just getting started.

Digital Brands completed a successful $10 million underwritten public offering last week, selling roughly 2.4 million shares of stock at $4.15 per share. Although the stock is trading approximately 30% lower from its offering price, wise investors recognize that with a public float of around 6.8 million shares, combined with warrants sold as p

add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 23388.31it/s]
2021-05-25 15:08:42,381 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing predictions to: predictions.json
2021-05-25 15:08:42,390 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing nbest to: nbest_predictions.json
add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 6845.98it/s]
2021-05-25 15:09:21,897 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing predictions to: predictions.json
2021-05-25 15:09:21,900 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing nbest to: nbest_predictions.json
add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 10512.04it/s]
2021-05-25 15:10:02,404 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing predictions to: predictions.json
2021-05-25 15:10:02,415 - 1620309 - transformers.data.metrics.squad_metrics - INFO - Writing nbest to: nbest_predictions.json

In [27]:
# dic1 = {'text':['x','y','z'],'age':[29,38]}
# dic2 = {'name':['x','y','z'],'address':[20,29,21]}
# df2 = pd.DataFrame(dic2)
# import pandas as pd 
# df1 = pd.DataFrame.from_dict(dic1,orient='index')
# df1 = df1.transpose()
# # df = [df1,df2]
# final = pd.concat([df1,df2],axis=1)
# final = final.reset_index(drop=True)
# # df.reset_index(drop=True, inplace=True)
# final